In [385]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout, Activation
#from keras.layers import Flatten
#from sklearn.metrics import roc_auc_score
#from io import StringIO
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [386]:
df1 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-9/qoedata.csv')
df2 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-10/qoedata.csv')
df3 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-11/qoedata.csv')
df4 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-12/qoedata.csv')
#df5 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-13/qoedata.csv')
df6 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-14/qoedata.csv')
#df7 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-15/qoedata.csv')
#df8 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-17/qoedata.csv')
df9 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-18/qoedata.csv')
#df10 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-20/qoedata.csv')
df11 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-21/qoedata.csv')
df12 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-22/qoedata.csv')
#df13 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-23/qoedata.csv')
df14 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-24/qoedata.csv')
df15 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-25/qoedata.csv')

df16 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-26/qoedata.csv')
df17 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-27/qoedata.csv')
#df18 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-28/qoedata.csv')
df19 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-29/qoedata.csv')
#df20 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-30/qoedata.csv')
df21 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-31/qoedata.csv')
#df22 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-32/qoedata.csv')
#df23 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-33/qoedata.csv')
df24 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-34/qoedata.csv')
#df25 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-35/qoedata.csv')
#df26 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-36/qoedata.csv')

In [387]:
df  = pd.concat([df1,df2,df3,df4,df6,df9,
                df11,df12,df14,df15,
                df16,df17,df19,df21,
               df24], axis = 0)
df = df[df.state4 != 0]
df25 = pd.read_csv('/home/zdf/桌面/20181231/data/bola-35/qoedata.csv')
df25 = df25[df25.state4 != 0]

In [388]:
df[0:1]

,Unnamed: 0,state4,state3,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,0,1,1,0.0,101363,101363,101363


In [389]:
df25[0:1]

,Unnamed: 0,state4,state3,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,0,1,1,0.0,252044,252044,252044


In [390]:
y_train = df['state4']
y_train = np.array(list(y_train))
y_train = preprocessing.label_binarize(y_train, classes = [1,2,3,4])

y_test = df25['state4']
y_test = np.array(list(y_test))
y_test = preprocessing.label_binarize(y_test, classes= [1,2,3,4])

In [391]:
y_train[0:3]
print(len(y_train))

35938


In [392]:
# 数据归一化，缩放特征数据
df_scaled = preprocessing.normalize(df[['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']])
df25_scaled = preprocessing.normalize(df25[['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']])

In [393]:
X_train = pd.DataFrame(df_scaled, columns = ['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']) 
X_test = pd.DataFrame(df25_scaled, columns = ['reqInterTime','downTraffic','downTrafficDiff','downTrafficDiffAbs']) 

In [394]:
X_train[0:3]

,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,0.000000,0.57735,0.57735,0.57735
1,0.000004,0.57735,0.57735,0.57735
2,0.000004,0.57735,0.57735,0.57735


In [395]:
X_test[0:5]

,reqInterTime,downTraffic,downTrafficDiff,downTrafficDiffAbs
0,0.000000e+00,0.57735,0.57735,0.57735
1,1.116680e-06,0.57735,0.57735,0.57735
2,9.887917e-07,0.57735,0.57735,0.57735
3,9.887917e-07,0.57735,0.57735,0.57735
4,2.473835e-06,0.57735,0.57735,0.57735


In [396]:
'''
tmp1 = {
        'request':X_train['reqInterTime'],
        'downTraffic':X_train['downTraffic'],
        'downTrafficDiff':X_train['downTrafficDiff'],
        'downTrafficDiffAbs':X_train['downTrafficDiffAbs']
        #'label':df_train['state4']
}
tmp2 = {
        'request':X_test['reqInterTime'],
        'downTraffic':X_test['downTraffic'],
        'downTrafficDiff':X_test['downTrafficDiff'],
        'downTrafficDiffAbs':X_test['downTrafficDiffAbs']
        #'label':df_test['state4']
}
'''

"\ntmp1 = {\n        'request':X_train['reqInterTime'],\n        'downTraffic':X_train['downTraffic'],\n        'downTrafficDiff':X_train['downTrafficDiff'],\n        'downTrafficDiffAbs':X_train['downTrafficDiffAbs']\n        #'label':df_train['state4']\n}\ntmp2 = {\n        'request':X_test['reqInterTime'],\n        'downTraffic':X_test['downTraffic'],\n        'downTrafficDiff':X_test['downTrafficDiff'],\n        'downTrafficDiffAbs':X_test['downTrafficDiffAbs']\n        #'label':df_test['state4']\n}\n"

In [397]:
'''
bufferStateRecongnize_train = pd.DataFrame(tmp1)
cols_train = list(bufferStateRecongnize_train)
#cols_train.insert(0, cols_train.pop(cols_train.index('label')))
#bufferStateRecongnize_train = bufferStateRecongnize_train.loc[:, cols_train]
'''

"\nbufferStateRecongnize_train = pd.DataFrame(tmp1)\ncols_train = list(bufferStateRecongnize_train)\n#cols_train.insert(0, cols_train.pop(cols_train.index('label')))\n#bufferStateRecongnize_train = bufferStateRecongnize_train.loc[:, cols_train]\n"

In [398]:
'''
bufferStateRecongnize_test = pd.DataFrame(tmp2)
cols_test = list(bufferStateRecongnize_test)
#cols_test.insert(0, cols_test.pop(cols_test.index('label')))
#bufferStateRecongnize_test = bufferStateRecongnize_test.loc[:, cols_test]
'''

"\nbufferStateRecongnize_test = pd.DataFrame(tmp2)\ncols_test = list(bufferStateRecongnize_test)\n#cols_test.insert(0, cols_test.pop(cols_test.index('label')))\n#bufferStateRecongnize_test = bufferStateRecongnize_test.loc[:, cols_test]\n"

In [399]:
X_train = X_train.values.astype('float32')
X_test = X_test.values.astype('float32')

look_back = 4
trainX = []
testX = []
trainY = []
testY = []

for i in range(len(X_train)-look_back-1):
    traintmp = X_train[i:(i+look_back),0:]
    trainX.append(traintmp)
    trainY.append(y_train[i+look_back])
    
trainX = np.reshape(trainX, (len(trainX), look_back, 4))
trainY = np.reshape(trainY, (len(trainY),1,4))

In [403]:
for i in range(len(X_test)-look_back-1):
    testmp = X_test[i:(i+look_back),0:]
    testX.append(testmp)
    testY.append(y_test[i+look_back])
    
testX = np.reshape(testX, (len(testX), look_back, 4))
testY = np.reshape(testY, (len(testY),1,4))

In [404]:
len(testX),len(testY)

(2395, 2395)

In [405]:
testX[0:1]

array([[[0.0000000e+00, 5.7735026e-01, 5.7735026e-01, 5.7735026e-01],
        [1.1166800e-06, 5.7735026e-01, 5.7735026e-01, 5.7735026e-01],
        [9.8879173e-07, 5.7735026e-01, 5.7735026e-01, 5.7735026e-01],
        [9.8879173e-07, 5.7735026e-01, 5.7735026e-01, 5.7735026e-01]]],
      dtype=float32)

In [406]:
testY[0:1]

array([[[1, 0, 0, 0]]])

In [409]:
#创建一个实例history
history = LossHistory()

model = Sequential()
model.add(LSTM(128, dropout = 0.2, activation='tanh', input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(Dense(trainY.shape[1],activation='relu'))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
model.fit(trainX, trainY, 
          nb_epoch = 5, 
          batch_size = 1, 
          verbose = 1,
          validation_data = (testX, testY),
          callbacks=[history]) 

score = model.evaluate(testX, testY, batch_size = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#绘制acc-loss曲线
history.loss_plot('epoch')

/home/zdf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ValueError: Error when checking target: expected dense_30 to have 2 dimensions, but got array with shape (35933, 1, 4)

auc

# 预测效果

In [529]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,mutual_info_score

In [530]:
test_preY = model.predict_proba(testX)

In [531]:
print('precision score：%f'%precision_score(testY,test_preY))
print('recall score：%f'%recall_score(testY,test_preY))
print('F1 score：%f'%f1_score(testY,test_preY))
print('AUC：%f'%roc_auc_score(testY,test_preY))

precision score：0.776565
recall score：1.000000
F1 score：0.874232
AUC：0.500000


# 滑动窗口预测

In [532]:
def predict_sequence_full(model, data, window_size):  
    predicted = []
    for i in range(len(data)):
        predicted.append(model.predict_proba(testX[i,:,:].reshape(1,testX[i,:,:].shape[0],testX[i,:,:].shape[1]))) 
    return predicted

In [533]:
y_pred = predict_sequence_full(model,testX,1)

In [534]:
testY.shape

(9985, 1)

In [535]:
y_pred = np.array(y_pred).reshape(len(y_pred),1)

In [536]:
print('precision score：%f'%precision_score(testY,y_pred))
print('recall score：%f'%recall_score(testY,y_pred))
print('F1 score：%f'%f1_score(testY,y_pred))
print('AUC：%f'%roc_auc_score(testY,y_pred))

precision score：0.776565
recall score：1.000000
F1 score：0.874232
AUC：0.500000


# 
# 
# 
# 
# 
# 

# 重新训练

In [377]:
from keras import backend as K
import tensorflow as tf

In [405]:
def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 1:]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX,dtype = 'float16'), np.array(dataY,dtype = 'int')

bola_dataset = bola_videoCatonRecongnize.values
bola_dataset = bola_dataset.astype('float16')

dynamic_dataset = dynamic_videoCatonRecongnize.values
dynamic_dataset = dynamic_dataset.astype('float16')
train = dynamic_dataset
test = bola_dataset

In [406]:
train[np.isinf(train)] = 65000
test[np.isinf(test)] = 65000

In [386]:
look_back = 5
trainx,trainy = create_dataset(train,look_back)
testx,testy = create_dataset(test,look_back)
trainx = np.reshape(trainx, (trainx.shape[0], look_back, trainx.shape[2]))
trainy = np.reshape(trainy, (trainy.shape[0], 1))
testx = np.reshape(testx, (testx.shape[0], look_back, testx.shape[2]))
testy = np.reshape(testy, (testy.shape[0], 1))

In [576]:
trainx[0]

array([[0.000e+00, 1.243e+00],
       [2.581e+04, 1.243e+00],
       [6.166e+04, 1.243e+00],
       [4.589e+04, 1.243e+00],
       [3.571e+04, 1.243e+00]], dtype=float16)

In [496]:
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

In [577]:
model1 = Sequential()
model1.add(LSTM(32, dropout = 0.2, activation='tanh', input_shape=(trainx.shape[1],trainx.shape[2])))
model1.add(Dense(trainy.shape[1],activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = [auc])
model1.fit(trainx, trainy, 
          nb_epoch = 20, 
          batch_size = 64, 
          verbose = 1,
          validation_data = (testx, testy))

score = model1.evaluate(testx, testy, batch_size = 64)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.5/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9872 samples, validate on 9990 samples
Epoch 1/20
9872/9872 [==============================] - 42s 4ms/step - loss: 0.6537 - auc: 0.2271 - val_loss: 0.5760 - val_auc: nan
Epoch 2/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5410 - auc: 0.4837 - val_loss: 0.5444 - val_auc: nan
Epoch 3/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5267 - auc: 0.4875 - val_loss: 0.5387 - val_auc: nan
Epoch 4/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5222 - auc: 0.5112 - val_loss: 0.5381 - val_auc: nan
Epoch 5/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5214 - auc: 0.5165 - val_loss: 0.5371 - val_auc: nan
Epoch 6/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5202 - auc: 0.5258 - val_loss: 0.5378 - val_auc: nan
Epoch 7/20
9872/9872 [==============================] - 10s 1ms/step - loss: 0.5202 - auc: 0.5274 - val_loss: 0.5362 - val_auc: nan
Epoch 8/20
9872/9872 [======

In [578]:
y_pred = model1.predict(testx)
y_pred = np.array(y_pred).reshape(len(y_pred))
testy = testy.reshape(len(testy))
print('AUC：%f'%roc_auc_score(testy,y_pred))

# y_pred = np.array([1 if i > 0.50 else 0 for i in range(len(y_pred))])
# print('precision score：%f'%precision_score(testy,y_pred))
# print('recall score：%f'%recall_score(testy,y_pred))
# print('F1 score：%f'%f1_score(testy,y_pred))

AUC：0.473074


In [579]:
len([i for i in range(len(testy)) if testy[i] < 0.5])

2236

In [580]:
[i for i in range(len(y_pred)) if y_pred[i] < 0.5]

[]

# lstm不手动追溯前面时刻点
auc：验证集nan 测试集0.68

In [660]:
test = test[:5000]

In [661]:
tr_x = train[:,1:]

In [662]:
tr_y = train[:,:1]

In [663]:
tt_x = test[:,1:]

In [664]:
tt_y = test[:,:1]

In [665]:
model2 = Sequential()
model2.add(Embedding(500, 128, input_length=2))
model2.add(LSTM(output_dim=128, activation='sigmoid', recurrent_activation='hard_sigmoid', unit_forget_bias=True))
model2.add(Dropout(0.5))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc])

model2.fit(tr_x, tr_y, 
          nb_epoch = 20, 
          batch_size = 64, 
          verbose = 1,
          validation_data = (tt_x, tt_y))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(unit_forget_bias=True, recurrent_activation="hard_sigmoid", activation="sigmoid", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9978 samples, validate on 5000 samples
Epoch 1/20
9978/9978 [==============================] - 43s 4ms/step - loss: 0.2070 - auc: nan - val_loss: 0.2009 - val_auc: nan
Epoch 2/20
9978/9978 [==============================] - 8s 767us/step - loss: 0.1521 - auc: nan - val_loss: 0.2013 - val_auc: nan
Epoch 3/20
9978/9978 [==============================] - 8s 783us/step - loss: 0.1476 - auc: nan - val_loss: 0.2039 - val_auc: nan
Epoch 4/20
9978/9978 [==============================] - 8s 800us/step - loss: 0.1476 - auc: nan - val_loss: 0.2018 - val_auc: nan
Epoch 5/20
9978/9978 [==============================] - 8s 784us/step - loss: 0.1476 - auc: nan - val_loss: 0.2023 - val_auc: nan
Epoch 6/20
9978/9978 [==============================] - 8s 794us/step - loss: 0.1468 - auc: nan - val_loss: 0.2020 - val_auc: nan
Epoch 7/20
9978/9978 [==============================] - 8s 790us/step - loss: 0.1465 - auc: nan - val_loss: 0.2038 - val_auc: nan
Epoch 8/20
9978/9978 [=====================

In [666]:
y_pred = model2.predict_proba(tt_x)
y_pred = np.array(y_pred).reshape(len(y_pred))
tt_y = tt_y.reshape(len(tt_y))
print('AUC：%f'%roc_auc_score(tt_y,y_pred))

# y_pred = np.array([1 if y_pred[i] > 0.50 else 0 for i in range(len(y_pred))])
# print('precision score：%f'%precision_score(tt_y,y_pred))
# print('recall score：%f'%recall_score(tt_y,y_pred))
# print('F1 score：%f'%f1_score(tt_y,y_pred))

AUC：0.684453


In [667]:
[i for i in range(len(y_pred)) if y_pred[i] <= 0.5]

[]

In [668]:
y_pred

array([0.8471982 , 0.8471982 , 0.8471982 , ..., 0.96054965, 0.96054965,
       0.96054965], dtype=float32)

# xgboost加上时序特征
auc:验证集0.60 测试集0.54

In [669]:
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix

In [725]:
test = test[:5000]
tr_x = train[:,1:]
tr_y = train[:,:1].flatten()
tt_x = test[:,1:]
tt_y = test[:,:1].flatten()

In [761]:
train

array([[0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00],
       ...,
       [1.000e+00, 0.000e+00, 4.512e+00],
       [1.000e+00, 5.736e+03, 4.512e+00],
       [1.000e+00, 0.000e+00, 4.512e+00]], dtype=float16)

In [726]:
list(tr_x[0:4,:].flatten())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [746]:
def reconstruct(data,lookback=5): # 对data数据集进行操作，datax包含时序信息
    datax = []
    datay = []
    for i in range(len(data)-look_back-1):
        a = list(data[i:(i+look_back), 1:].flatten())
        datax.append(a)
        b = data[i][0]
        datay.append(b)
    return np.array(datax),np.array(datay)

In [753]:
tr_x,tr_y = reconstruct(train)
tt_x,tt_y = reconstruct(test)

In [768]:
tr_x

array([[   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       ...,
       [   0.  ,    4.51,    0.  , ...,    4.51, 2868.  ,    4.51],
       [   0.  ,    4.51, 4304.  , ...,    4.51,    0.  ,    4.51],
       [4304.  ,    4.51, 2868.  , ...,    4.51,    0.  ,    4.51]],
      dtype=float16)

In [762]:
tr_y

array([0., 0., 0., ..., 1., 1., 1.], dtype=float16)

In [769]:
xgb1 = XGBClassifier(
                     objective= 'binary:logistic', 
                     nthread=4, 
                     max_depth=2,
                     min_child_weight=4, 
                     learning_rate =0.1, 
                     n_estimators=140, 
                     gamma=0, 
                     subsample=0.8,
                     colsample_bytree=0.8,    
                     scale_pos_weight=1, 
                     seed=27,
                     
                    )

In [770]:
xgb1.fit(tr_x,tr_y,eval_set=[(tt_x,tt_y)],eval_metric='auc')

[0]	validation_0-auc:0.636229
[1]	validation_0-auc:0.673296
[2]	validation_0-auc:0.673228
[3]	validation_0-auc:0.673152
[4]	validation_0-auc:0.673143
[5]	validation_0-auc:0.673133
[6]	validation_0-auc:0.739203
[7]	validation_0-auc:0.738876
[8]	validation_0-auc:0.737986
[9]	validation_0-auc:0.737831
[10]	validation_0-auc:0.739409
[11]	validation_0-auc:0.739882
[12]	validation_0-auc:0.73942
[13]	validation_0-auc:0.739564
[14]	validation_0-auc:0.740086
[15]	validation_0-auc:0.741567
[16]	validation_0-auc:0.741541
[17]	validation_0-auc:0.741453
[18]	validation_0-auc:0.741598
[19]	validation_0-auc:0.741551
[20]	validation_0-auc:0.741897
[21]	validation_0-auc:0.741632
[22]	validation_0-auc:0.742058
[23]	validation_0-auc:0.74139
[24]	validation_0-auc:0.741651
[25]	validation_0-auc:0.74126
[26]	validation_0-auc:0.741536
[27]	validation_0-auc:0.741306
[28]	validation_0-auc:0.741317
[29]	validation_0-auc:0.741319
[30]	validation_0-auc:0.741296
[31]	validation_0-auc:0.732453
[32]	validation_0-auc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=4, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [771]:
pred = xgb1.predict(tt_x)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [772]:
roc_auc_score(pred,tt_y)

0.535770703023542

In [773]:
list(tt_y).count(0)

268

In [774]:
confusion_matrix(tt_y,pred)

array([[  24,  244],
       [ 172, 4549]])